In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from urllib.request import urlopen

import socks
import socket
import requests
import pandas as pd
import numpy as np
import time

In [2]:
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
old_socket = socket.socket # saving old socket info 
socket.socket = socks.socksocket
def getaddrinfo(*args):
    return [(socket.AF_INET, socket.SOCK_STREAM, 6, '', (args[0], args[1]))]
socket.getaddrinfo = getaddrinfo

## 1) Scraping REvil

In [ ]:
REvil_link = "http://dnpscnbaix6nkwvystl3yxglz7nteicqrou3t75tpcc5532cztc46qyd.onion"
res = requests.get(REvil_link)

In [ ]:
# Putting all weblinks into a list 
links = [link.get('href') for link in soup.find_all('a', href=True)]
links = list(filter(None, links))

webpage_links = [] 
for l in links:
    if "page" in l:
        l = REvil_link + l
        webpage_links.append(l)
        
webpage_links = webpage_links[1:-1] # remvoving first and last element from list

In [ ]:
# Getting list of victims from REvil
REvilHeader_list = []
for link in webpage_links:
    res = requests.get(link)
    soup = BeautifulSoup(res.content, 'html.parser')
    posts = soup.find_all(class_ = "blog-post-title")
    
    for post in posts:
        title = post.find('a')
        REvilHeader_list.append(title.text)

In [ ]:
# Placing victims into a DataFrame 
df = pd.DataFrame(columns=['REvil Headers', 'Company Name', 'Industry', 'Headquarters'])
df['REvil Headers'] = REvilHeader_list

df

In [ ]:
# Using Selenium to automate finding industries
socket.socket = old_socket # reuse the original socket for selenium
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()

In [ ]:
# glassdoor search
company_list = []
industry_list = []
location_list = []
number = 0

for header in REvilHeader_list: 
    try:
        driver.get('https://www.google.com')
        search = driver.find_element(By.NAME, 'q')
        search.send_keys(header + ' glassdoor overview')
        search.send_keys(Keys.RETURN)
        glassdoor_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        glassdoor_page.click()
        
        if "glassdoor" in driver.current_url:
            company = driver.find_element_by_tag_name('h1')
            company_list.append(company.text)
            
            pagesource = driver.page_source
            soup = BeautifulSoup(pagesource, 'html.parser')
            labels = soup.find_all('label')
            for label in labels:
                if (label.text == "Industry:"):
                    industry = label.next_sibling.text
                    industry_list.append(industry)
                    break
            else:
                industry_list.append(np.nan)
                
            for label in labels:
                if (label.text == "Headquarters:"):
                    location = label.next_sibling.text
                    location_list.append(location)
                    break
            else:
                location_list.append(np.nan)
        else:
            company_list.append(np.nan)
            industry_list.append(np.nan)
            location_list.append(np.nan)
    except Exception as e:
        company_list.append(np.nan)
        industry_list.append(np.nan)
        location_list.append(np.nan)
        print(e)

    print(str(number) + ") " + header)
    number = number + 1
    
driver.quit()